<a href="https://colab.research.google.com/github/DangMinh21/CS114.L22.KHCL/blob/main/Recognizing_hand_written_digits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recognizing hand-written digits

This example shows how scikit-learn can be used to recognize images of
hand-written digits, from 0-9.

In [ ]:
print(__doc__)

# Author: Gael Varoquaux <gael dot varoquaux at normalesup dot org>
# License: BSD 3 clause

# Standard scientific Python imports
import matplotlib.pyplot as plt

# Import datasets, classifiers and performance metrics
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split

## Digits dataset

The digits dataset consists of 8x8
pixel images of digits. The ``images`` attribute of the dataset stores
8x8 arrays of grayscale values for each image. We will use these arrays to
visualize the first 4 images. The ``target`` attribute of the dataset stores
the digit each image represents and this is included in the title of the 4
plots below.

Note: if we were working from image files (e.g., 'png' files), we would load
them using :func:`matplotlib.pyplot.imread`.

In [ ]:
digits = datasets.load_digits()

_, axes = plt.subplots(nrows=1, ncols=4, figsize=(10, 3))
for ax, image, label in zip(axes, digits.images, digits.target):
    ax.set_axis_off()
    ax.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    ax.set_title('Training: %i' % label)

## Classification

To apply a classifier on this data, we need to flatten the images, turning
each 2-D array of grayscale values from shape ``(8, 8)`` into shape
``(64,)``. Subsequently, the entire dataset will be of shape
``(n_samples, n_features)``, where ``n_samples`` is the number of images and
``n_features`` is the total number of pixels in each image.

We can then split the data into train and test subsets and fit a support
vector classifier on the train samples. The fitted classifier can
subsequently be used to predict the value of the digit for the samples
in the test subset.

In [ ]:
# flatten the images
n_samples = len(digits.images)
data = digits.images.reshape((n_samples, -1))

# Create a classifier: a support vector classifier
clf = svm.SVC(gamma=0.001)

# Split data into 50% train and 50% test subsets
X_train, X_test, y_train, y_test = train_test_split(
    data, digits.target, test_size=0.5, shuffle=False)

# Learn the digits on the train subset
clf.fit(X_train, y_train)

# Predict the value of the digit on the test subset
predicted = clf.predict(X_test)

Below we visualize the first 4 test samples and show their predicted
digit value in the title.

In [ ]:
_, axes = plt.subplots(nrows=1, ncols=4, figsize=(10, 3))
for ax, image, prediction in zip(axes, X_test, predicted):
    ax.set_axis_off()
    image = image.reshape(8, 8)
    ax.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    ax.set_title(f'Prediction: {prediction}')

In [ ]:
print(f"Classification report for classifier {clf}:\n"
      f"{metrics.classification_report(y_test, predicted)}\n")

In [ ]:
disp = metrics.plot_confusion_matrix(clf, X_test, y_test)
disp.figure_.suptitle("Confusion Matrix")
print(f"Confusion matrix:\n{disp.confusion_matrix}")

plt.show()

# Phân tích trước khi thực hiện
### Đặc điểm ảnh trong bộ dataset
* kích thước: 8*8 grayscale
* giá trị mỗi pixel nằm trong khoảng 0 - 15

### Đặc điểm ảnh chụp được
* Kích thước: 480*640 RBL
* giá trị mỗi pixel nằm trong khoảng 0 - 255

Sau khi chụp được hình từ webcam, thì chuyển đổi để ảnh chụp được có đặc điểm giống ảnh trong bộ dataset sau đó thực hiện predict.

#WEBCAM

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

#IMPORT THƯ VIỆN

In [ ]:
import cv2 as cv
import numpy as np

#CHỤP HÌNH SỐ ĐỂ NHẬN DẠNG
* Chụp 6 hình lưu các số từ 1 đến 6 vào list images
* tạo bộ target [1,2,3,4,5,6] tương ứng với nội dung ảnh

In [ ]:
images = []
target = [1, 2, 3, 4, 5, 6]
from IPython.display import Image
temp = 0
while(temp < 6):
  try:
    filename = take_photo()
    print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
    display(Image(filename))
  except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
    print(str(err))
  # chuyển ảnh về dạng grayscale rồi lưu vào images
  image = cv.imread(filename, 0)
  images.append(image)
  temp +=1

## 6 số chụp được từ webcam laptop

In [ ]:
_, axes = plt.subplots(nrows=1, ncols=6, figsize=(200, 30))
for ax, image, label in zip(axes, images, target):
    ax.set_axis_off()
    ax.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    ax.set_title('target: %i' % label)

In [ ]:
#chuyển images về ndarray để dễ sử lí
images = np.array(images)

In [ ]:
print(images.shape)

In [ ]:
# thực hiện chuyển ảnh từ kích thước (480, 640) về kích thước (8, 8)
Images = []
for i in range(6):
  temp = cv.resize(images[i], (8, 8))
  Images.append(temp)

In [ ]:
print(Images.shape)

In [ ]:
# xuất 6 ảnh sau khi thực hiện chuyển đổi
Images = np.array(Images)
_, axes = plt.subplots(nrows=1, ncols=6, figsize=(200, 30))
for ax, image, label in zip(axes, Images, target):
    ax.set_axis_off()
    ax.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    ax.set_title('target: %i' % label)

In [ ]:
# đưa ảnh về (64, 1) để thực hiện predict
# đồng thời đưa giá trị mỗi pixel về trong khoảng (0, 15)
images_flat = Images.reshape(6, 64)
for i in range(len(images_flat)):
  images_flat[i] = images_flat[i]/255*15


In [ ]:
#thực hiện predict
y_pred = clf.predict(images_flat)

In [ ]:
print(y_pred)

In [ ]:
# 6 ảnh sau khi thực hiện predict cùng label được predict
_, axes = plt.subplots(nrows=1, ncols=6, figsize=(10, 3))
for ax, image, prediction in zip(axes, images_flat, y_pred):
    ax.set_axis_off()
    image = image.reshape(8, 8)
    ax.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    ax.set_title(f'Prediction: {prediction}')

## kết quả so sánh sau khi predict

In [ ]:
print(f"Classification report for classifier {clf}:\n"
      f"{metrics.classification_report(target, y_pred)}\n")

In [ ]:
disp = metrics.plot_confusion_matrix(clf, images_flat, target)
disp.figure_.suptitle("Confusion Matrix")
print(f"Confusion matrix:\n{disp.confusion_matrix}")

plt.show()

# TRẢ LỜI CÂU HỎI
### Kết quả chạy thực tế với webcam của laptop và giấy viết tay của các bạn có tốt không?
>kết quả không được tốt, cụ thể 6 ảnh chụp được ở trên không predict đúng ảnh nào.
### Ủa mà nhìn vào đâu để kết luận là tốt hay không?
>nhìn vào kết quả so sánh thấy được độ chính xác không tốt.
### Nếu không tốt thì lý do tại sao?
* webcam laptop chụp hình có chất lượng không tốt, điều kiện ánh sáng, môi trường không phù hợp.
* sau khi chuyển ảnh chụp được từ webcam sang ảnh có đặc điểm giống ảnh trong dataset thì đã mất đi nhiều thông tin, xử kí ảnh không tốt.
* Model bị overfit với bộ dataset.

### Nếu tốt thì tiếp theo có thể dùng model digits recognition này vào các bài toán phức tạp hơn như nhận dạng biển số nhà không?
> Không sử dụng model này vào thực tế nhận dạng biển số được. Để sử dụng model trong thực tế cần cập nhập lại bộ dataset cho phù hợp cộng với sử đụng các kĩ thuật để chuyển đổi ảnh cho phù hợp